## Browsing elephant modules and functions

In [1]:
import requests

In [2]:
requests.get('http://localhost:5000/api').json()

['cell_assembly_detection',
 'change_point_detection',
 'conversion',
 'cubic',
 'current_source_density',
 'current_source_density_src',
 'gpfa',
 'kernels',
 'neo_tools',
 'phase_analysis',
 'signal_processing',
 'spade',
 'spade_src',
 'spectral',
 'spike_train_correlation',
 'spike_train_dissimilarity',
 'spike_train_generation',
 'spike_train_surrogates',
 'sta',
 'statistics',
 'unitary_event_analysis',
 'utils',
 'waveform_features']

In [3]:
requests.get('http://localhost:5000/api/statistics').json()

['SpikeTrain',
 'complexity_pdf',
 'conv',
 'cost_function',
 'cv',
 'cv2',
 'division',
 'fanofactor',
 'fftkernel',
 'ilogexp',
 'instantaneous_rate',
 'isi',
 'kernels',
 'logexp',
 'lv',
 'mean_firing_rate',
 'neo',
 'nextpow2',
 'np',
 'pq',
 'print_function',
 'scipy',
 'sskernel',
 'time_histogram',
 'warnings']

## POST requests

### Format

The main payload - kwargs for an elephant function - must be provided in `data` key of json POST request.

```python
response = requests.post('http://localhost:5000/api/<module>/<function>', json=dict(
    data=dict(...),  # required
    units=dict(time='ms', amplitude='mV', rate='Hz'),  # optional
    t_stop=120.57,  # optional; in the specified units; default is None
    t_start=2.3,  # optional; in the specified units; default is 0
))
```

If no units are provied, the default units are used:
* `ms` for time
* `mV` for the amplitude of an analog signal
* `Hz` for rates


## Examples

### Inter-spike interval

In [4]:
response = requests.post('http://localhost:5000/api/statistics/isi', json=dict(
    data=dict(spiketrain=[1,2,3,4,11])
))

In [5]:
response.json()

[1.0, 1.0, 1.0, 7.0]

The resulting array `[1.0, 1.0, 1.0, 7.0]` is inter-spike intervals with the (default) units of `ms`.


#### Customize units, t_start, and t_stop, shared within the request

If you want to explicitly specify the units:

In [6]:
response = requests.post('http://localhost:5000/api/statistics/isi', json=dict(
    data=dict(spiketrain=[1,2,3,4,11]),
    units=dict(time='s')  # specify that all units are in secods
))

response.json()

[1.0, 1.0, 1.0, 7.0]

No difference, because currently elephant-sever does NOT output the units used - it assumes that the clients know what the units were. This behaviour can be easily changed.


#### Custom arguments to specific elephant functions

You can customize calls to elephant functions. Doing so requires filling all the necessary kwargs in json POST request:

In [7]:
response = requests.post('http://localhost:5000/api/statistics/isi', json=dict(
    data=dict(
        spiketrain=dict(times=[1,2,3,4,11], units='s', t_stop=27)
    ),
    units=dict(time='ms')
))

response.json()  # ISI in ms

[1000.0, 1000.0, 1000.0, 7000.0]

### CV2, LV

NEST users are typically interested in the coefficient of variation and local variation analyses, and the request is:

In [8]:
response = requests.post('http://localhost:5000/api/statistics/cv2', json=dict(
    data=dict(v=[1,2,3,4,11])
))

response.json()

0.5714285714285714

## Invalid requests

If you forgot to specify `t_stop` in the previous example:

In [9]:
bad_response = requests.post('http://localhost:5000/api/statistics/isi', json=dict(
    data=dict(
        spiketrain=dict(times=[1,2,3,4,11], units='s')
    ),
    units=dict(time='ms')
))

print(bad_response)
print()
print(bad_response.text)

<Response [400]>

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>400 Bad Request</title>
<h1>Bad Request</h1>
<p>DeserializeError('TypeError(&quot;__new__() missing 1 required positional argument: \'t_stop\'&quot;)')</p>



However, for some reasons, you cannot decode the error response:

In [10]:
bad_response.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)